## Desafio 1

Construir un modelo de inteligencia artificial capaz de clasificar noticias de prensa según su categoría temática principal.

category: sociedad, salud, politica, medioambiente, internacional, entretenimiento, economia, deportes, cultura, cienciatecnologia


In [1]:
# Librerias
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import spacy
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from geopy.geocoders import Nominatim
from transformers import pipeline
import re
from geopy.geocoders import Nominatim

#### Preparacion de dataframes


In [2]:
# Cargar el dataset de entrenamiento
df = pd.read_csv('./train_data.csv')

df_train = df.sample(n=5000, random_state=42)

# Barajar el DataFrame
df_train = df_train.sample(frac=1).reset_index(drop=True)

def format_df(df):
    # Convertir la columna de fechas al formato deseado
    df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')

    # Extraer la fecha en el formato YYYY-MM-DD
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')

    # Mostrar las primeras filas del DataFrame
    display(df)
    return df

df_train = format_df(df_train)

,date,id_news,media_outlet,title,text,url,year,clase
0,2022-08-03,"49,079,987",latercera,“El estallido social no fue un capricho”: la r...,Una reflexión sobre el estallido social del 18...,https://www.latercera.com/politica/noticia/el-...,"2,022",medio_ambiente
1,2022-04-28,"49,552,270",emol,Dólar cae este viernes y cierra volátil semana...,El dólar terminó la jornada de este viernes an...,https://www.emol.com/noticias/Economia/2022/04...,"2,022",economia
2,2022-04-12,"47,162,020",laserenaonline,Con firma de convenio entre Gobierno Regional ...,"La iniciativa, que es parte de la cartera de p...",https://laserenaonline.cl/2022/04/13/con-firma...,"2,022",economia
3,2022-06-27,"51,295,378",emol,OTAN celebra una cumbre con la mirada puesta e...,#emolmlt-notafotos{float: none; clear: bo...,https://www.emol.com/noticias/Internacional/20...,"2,022",medio_ambiente
4,2022-08-13,"51,155,822",elciudadano,«El ecodiseño busca crear un mejor vivir para ...,El 5to Congreso Latinoamericano de Ecodiseño “...,https://www.elciudadano.com/chile/el-ecodiseno...,"2,022",salud
...,...,...,...,...,...,...,...,...
4995,2022-02-21,"50,354,211",biobiochile,Gabriel Torres vuelve al fútbol chileno: Depor...,"El delantero panameño Gabriel Torres, que bril...",https://www.biobiochile.cl/noticias/deportes/f...,"2,022",deportes
4996,2022-04-29,"50,013,786",emol,El escenario económico y el acuerdo para siste...,blockquote.cita-aetv-pri {background-color: #f...,https://www.emol.com/noticias/Nacional/2022/04...,"2,022",economia
4997,2022-11-07,"49,558,687",eldinamo,La revista People eligió a Chris Evans como el...,Contento de que su mamá pueda presumir de ser ...,https://www.eldinamo.cl/entretencion/2022/11/0...,"2,022",ciencia
4998,2021-07-04,"22,069,519",elciudadano,Proyecto mexicano evitaría contagio de enferme...,Por la Redacción Ciudad de México. El Centro d...,https://www.elciudadano.com/mexico/proyecto-me...,"2,021",tecnologia


In [3]:
#categorias unicas
categorias = df_train['clase'].unique()
print(categorias)

# Conteo de categorias totales en el dataset de entrenamiento
df_train.groupby('clase').count()

['medio_ambiente' 'economia' 'salud' 'internacional' 'entretenimiento'
 'politica' 'accidentes' 'educacion' 'tecnologia' 'deportes' 'ciencia']


,date,id_news,media_outlet,title,text,url,year
clase,,,,,,,
accidentes,459,459,459,459,459,459,459
ciencia,470,470,470,470,470,470,470
deportes,432,432,432,432,432,432,432
economia,457,457,457,457,457,457,457
educacion,462,462,462,462,462,462,462
entretenimiento,442,442,442,442,442,442,442
internacional,448,448,448,448,448,448,448
medio_ambiente,477,477,477,477,477,477,477
politica,395,395,395,395,395,395,395


#### Procesamiento


In [4]:
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Procesar el texto con spaCy
    doc = nlp(text)
    
    # Lemmatizar y eliminar stop words y puntuación
    lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    # Filtrar espacios vacíos y devolver los tokens lematizados
    lemmatized_tokens = [token for token in lemmatized_tokens if token.strip()]
    return ' '.join(lemmatized_tokens)

tqdm.pandas(desc="Procesando texto")
df_train['tokenizacions'] = df_train['text'].progress_apply(preprocess_text)

Procesando texto: 100%|██████████| 5000/5000 [11:19<00:00,  7.36it/s]


In [5]:
# Separar el dataset en datos de entrenamiento y prueba
X = df_train['tokenizacions']  # Utilizamos el texto preprocesado
y = df_train['clase']  # Las etiquetas

# Dividir el dataset en conjunto de entrenamiento y prueba con estratificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto TfidfVectorizer
vectorizer = TfidfVectorizer()

# Ajustar el vectorizador y transformar los datos de entrenamiento
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Entrenamiento del modelo
model = LogisticRegression(max_iter=1000)  # Aumentamos max_iter si es necesario
model.fit(X_train_vectorized, y_train)

LogisticRegression(max_iter=1000)

In [6]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_vectorized)

# Comparar las predicciones con las etiquetas reales
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Matriz de confusión:
[[61 12  6  2  1  5  3  4  0  3  1]
 [ 5 38  1  3  2  4  5  3  3  6  6]
 [ 2  7 35  4  7 17  0  5  1  7  7]
 [ 2  3  4 49  5  2  0 13  1  8  3]
 [ 4  4 11  2 43  8  0  5  5  6  9]
 [ 3 11  6  9  4 33  3  9  2  2  2]
 [ 0  1  0  2  1  1 82  1  2  2  1]
 [ 4  3  1  4  2  4  4 60  1  4 12]
 [ 2  5  1  2  2  2 10  2 44  1  1]
 [ 1 12  2 16  7  9  1 12  3 34  5]
 [ 1  5  4 11 12  6  2  5  2  3 46]]

Reporte de clasificación:
                 precision    recall  f1-score   support

     accidentes       0.72      0.62      0.67        98
        ciencia       0.38      0.50      0.43        76
       deportes       0.49      0.38      0.43        92
       economia       0.47      0.54      0.51        90
      educacion       0.50      0.44      0.47        97
entretenimiento       0.36      0.39      0.38        84
  internacional       0.75      0.88      0.81        93
 medio_ambiente       0.50      0.61      0.55        99
       politica       0.69      0.61     

In [7]:
# Definir el nuevo documento
new_documents = ['El Ministerio de Salud ha informado sobre un nuevo brote de infecciones respiratorias que afecta a varias regiones del país. Los expertos recomiendan seguir usando mascarillas en espacios cerrados y continuar con las medidas de higiene para evitar la propagación del virus.']

# Preprocesar el nuevo documento con spaCy
new_documents_preprocessed = [preprocess_text(doc) for doc in new_documents]

# Transformar el nuevo documento a la representación TF-IDF
new_features = vectorizer.transform(new_documents_preprocessed)

# Hacer predicciones con el modelo entrenado
predictions = model.predict(new_features)  # model es el modelo entrenado previamente (LogisticRegression en este caso)

# Imprimir las predicciones
print(predictions)

['salud']


### Desafio 2

Geolocalizar eventos en noticias

Utilizando técnicas de tratamiento automático del lenguaje, el objetivo del desafío es estructurar la información de una noticia de la manera siguiente:

- event: principal evento descrito en la noticia
- category: categoria temática del evento
- address: dirección dónde occurió el evento en formato: calle número, comuna, país
- latitud: latitud del evento
- longitud: longitud del evento

Resultado: El formato del archivo CSV output que entregar contiene las columnas siguientes: id_news, event, category, address, latitud, longitud


In [8]:
# Cargar el dataset de entrenamiento
df_d2 = pd.read_csv('./dataset_agosto2024.csv')

# Muestra aleatoria de 2000 registros
df_geo = df_d2.sample(n=2000, random_state=42)

# quitar las filas con la columa text == "empty"
df_geo = df_geo[df_geo['text'] != '(empty)']

df_geo = format_df(df_geo)

,date,media_outlet,text,url,title,id_news
10404,2024-08-15,biobiochile,Los reconocidos guitarristas Horacio Salinas y...,https://www.biobiochile.cl/noticias/artes-y-cu...,Horacio Salinas y Romilio Orellana unen sus tr...,53.878.712
20026,2024-08-01,lanacion,El Gobierno suspendió las clases en todos los...,https://www.lanacion.cl/suspenden-clases-en-la...,Suspenden clases en la RM y otras nueve region...,53.412.225
2043,2024-08-27,latercera,"El delegado de la Región Metropolitana, Gonzal...",https://www.latercera.com/nacional/noticia/se-...,¿Se cierra o no se cierra el Teatro Caupolicán...,53.558.369
11508,2024-08-13,biobiochile,"Dos adolescentes de 15 y 16 años, este último ...",https://www.biobiochile.cl/noticias/nacional/r...,Detienen a dos adolescentes por robo de especi...,53.877.607
8398,2024-08-18,biobiochile,"Según el pronóstico del tiempo, se esperan llu...",https://www.biobiochile.cl/noticias/servicios/...,"Anuncian lluvias, granizos, vientos sobre 70 k...",53.560.642
...,...,...,...,...,...,...
12832,2024-08-11,cnnchile,Organizaciones no gubernamentales como Espacio...,https://www.cnnchile.com/mundo/bloqueo-redes-s...,Bloqueo a redes sociales afectan el libre fluj...,53.427.424
17687,2024-08-05,biobiochile,Deportes Puerto Montt demanda a la ANFP por in...,https://www.biobiochile.cl/noticias/deportes/f...,Deportes Puerto Montt lleva a la ANFP a la Jus...,53.878.826
1624,2024-08-28,biobiochile,La FIFA encargó a una startup chilena la misió...,https://www.biobiochile.cl/noticias/ciencia-y-...,Más goles e ídolas: una IA reveló cómo sería e...,53.879.318
3310,2024-08-26,emol,El capitán se va. El retiro de Claudio Bravo ...,https://www.emol.com/noticias/Deportes/2024/08...,El homenaje que planea la ANFP y los especiale...,53.564.989


In [9]:
# utilizar modelo anterior para predecir la clase de los textos en el dataset
df_geo['tokenizacions'] = df_geo['text'].progress_apply(preprocess_text)

# Transformar los datos de texto a vectores TF-IDF
X_geo = df_geo['tokenizacions']
X_geo_vectorized = vectorizer.transform(X_geo)

# Realizar predicciones con el modelo entrenado
y_geo_pred = model.predict(X_geo_vectorized)

# Agregar las predicciones al DataFrame
df_geo['category'] = y_geo_pred
df_geo

Procesando texto: 100%|██████████| 1810/1810 [03:24<00:00,  8.86it/s]


,date,media_outlet,text,url,title,id_news,tokenizacions,category
10404,2024-08-15,biobiochile,Los reconocidos guitarristas Horacio Salinas y...,https://www.biobiochile.cl/noticias/artes-y-cu...,Horacio Salinas y Romilio Orellana unen sus tr...,53.878.712,Los reconocido guitarrista Horacio Salinas y R...,entretenimiento
20026,2024-08-01,lanacion,El Gobierno suspendió las clases en todos los...,https://www.lanacion.cl/suspenden-clases-en-la...,Suspenden clases en la RM y otras nueve region...,53.412.225,El Gobierno suspendió las clase en todo los es...,educacion
2043,2024-08-27,latercera,"El delegado de la Región Metropolitana, Gonzal...",https://www.latercera.com/nacional/noticia/se-...,¿Se cierra o no se cierra el Teatro Caupolicán...,53.558.369,El delegado de la Región Metropolitana Gonzalo...,entretenimiento
11508,2024-08-13,biobiochile,"Dos adolescentes de 15 y 16 años, este último ...",https://www.biobiochile.cl/noticias/nacional/r...,Detienen a dos adolescentes por robo de especi...,53.877.607,Dos adolescente de 15 y 16 año este último con...,ciencia
8398,2024-08-18,biobiochile,"Según el pronóstico del tiempo, se esperan llu...",https://www.biobiochile.cl/noticias/servicios/...,"Anuncian lluvias, granizos, vientos sobre 70 k...",53.560.642,Según el pronóstico del tiempo se esperan lluv...,medio_ambiente
...,...,...,...,...,...,...,...,...
12832,2024-08-11,cnnchile,Organizaciones no gubernamentales como Espacio...,https://www.cnnchile.com/mundo/bloqueo-redes-s...,Bloqueo a redes sociales afectan el libre fluj...,53.427.424,Organizaciones gubernamentale como Espacio Púb...,internacional
17687,2024-08-05,biobiochile,Deportes Puerto Montt demanda a la ANFP por in...,https://www.biobiochile.cl/noticias/deportes/f...,Deportes Puerto Montt lleva a la ANFP a la Jus...,53.878.826,Deportes Puerto Montt demanda la ANFP por inde...,deportes
1624,2024-08-28,biobiochile,La FIFA encargó a una startup chilena la misió...,https://www.biobiochile.cl/noticias/ciencia-y-...,Más goles e ídolas: una IA reveló cómo sería e...,53.879.318,La FIFA encargó una startup chilena la misión ...,deportes
3310,2024-08-26,emol,El capitán se va. El retiro de Claudio Bravo ...,https://www.emol.com/noticias/Deportes/2024/08...,El homenaje que planea la ANFP y los especiale...,53.564.989,El capitán se va El retiro de Claudio Bravo es...,deportes


In [10]:
# Cargar el modelo de lenguaje natural en español
nlp = spacy.load('es_core_news_sm')

# pipeline de pregunta-respuesta
qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")


/opt/miniconda3/envs/py_data/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
def extract_info(row):
    # Procesar el texto con spaCy
    text = row['text']
    doc = nlp(text)
    
    # Definir la pregunta para el modelo de QA para identificar el tipo de evento
    question_event = "¿De qué tipo de evento habla la noticia?"
    qa_response_event = qa_model(question=question_event, context=text)
    event = qa_response_event['answer']  # Extraer solo el texto de la respuesta

    # Usar el modelo QA para preguntar sobre el lugar del evento
    question_place = "¿Dónde ocurrió el evento?"
    qa_response_place = qa_model(question=question_place, context=text)
    place = qa_response_place['answer']  # Extraer la respuesta sobre el lugar

    # Usar spaCy para buscar entidades geográficas (GPE y LOC)
    addresses = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    
    # Si no se identificaron direcciones claras, usar el modelo de QA para ayudar a refinar la búsqueda
    if not addresses:
        question_address = "¿Cuál es la dirección exacta del evento?"
        qa_response_address = qa_model(question=question_address, context=text)
        addresses.append(qa_response_address['answer'])

    # Imprimir los resultados para verificar
    # print(f"Evento: {event}")
    # print(f"Lugar: {place}")
    # print(f"Direcciones encontradas: {addresses}")

    # Retornar los datos extraídos
    return event, place


# Aplicar la función a cada fila del DataFrame
tqdm.pandas(desc="Procesando texto")
#df_geo[['event', 'address']] = df_geo.progress_apply(extract_info, axis=1, result_type='expand')

### Para hacer las pruebas con menos datos descomentar las siguientes lineas y cambiar el dataframe utilizado despues

filtered_rows = []

# Iterar sobre las primeras 10 filas para depuración
for i in range(500):
    row = df_geo.iloc[i]
    
    # Extraer la información de evento y dirección
    event, address = extract_info(row)

    # Obtener el índice real de la fila actual
    index = df_geo.index[i]
    
    # Asignar los valores extraídos al DataFrame usando loc con el índice real
    df_geo.loc[index, 'event'] = event
    df_geo.loc[index, 'address'] = address  # Esto actualiza la dirección correctamente
    
    # Guardar la fila actualizada en la lista
    filtered_rows.append(df_geo.loc[index])

# Crear un nuevo DataFrame con las filas filtradas
df_filtered = pd.DataFrame(filtered_rows)

# Mostrar el nuevo DataFrame
display(df_filtered)



,date,media_outlet,text,url,title,id_news,tokenizacions,category,event,address
10404,2024-08-15,biobiochile,Los reconocidos guitarristas Horacio Salinas y...,https://www.biobiochile.cl/noticias/artes-y-cu...,Horacio Salinas y Romilio Orellana unen sus tr...,53.878.712,Los reconocido guitarrista Horacio Salinas y R...,entretenimiento,un concierto único.,Sala SCD Plaza Egaña.
20026,2024-08-01,lanacion,El Gobierno suspendió las clases en todos los...,https://www.lanacion.cl/suspenden-clases-en-la...,Suspenden clases en la RM y otras nueve region...,53.412.225,El Gobierno suspendió las clase en todo los es...,educacion,estragos causados por el sistema frontal.,Región de Coquimbo
2043,2024-08-27,latercera,"El delegado de la Región Metropolitana, Gonzal...",https://www.latercera.com/nacional/noticia/se-...,¿Se cierra o no se cierra el Teatro Caupolicán...,53.558.369,El delegado de la Región Metropolitana Gonzalo...,entretenimiento,"concierto,","Teatro Caupolicán,"
11508,2024-08-13,biobiochile,"Dos adolescentes de 15 y 16 años, este último ...",https://www.biobiochile.cl/noticias/nacional/r...,Detienen a dos adolescentes por robo de especi...,53.877.607,Dos adolescente de 15 y 16 año este último con...,ciencia,"robo con violencia,",calle Lynch a la altura del Destacamento Tuca...
8398,2024-08-18,biobiochile,"Según el pronóstico del tiempo, se esperan llu...",https://www.biobiochile.cl/noticias/servicios/...,"Anuncian lluvias, granizos, vientos sobre 70 k...",53.560.642,Según el pronóstico del tiempo se esperan lluv...,medio_ambiente,"tormentas eléctricas,","zona centro y sur,"
...,...,...,...,...,...,...,...,...,...,...
13499,2024-08-10,elcomercio,"/ ElucidarioProvee contexto, definición y deta...",https://elcomercio.pe/respuestas/que/la-respue...,La respuesta de Carlos Zambrano a las fuertes ...,53.406.006,ElucidarioProvee contexto definición y detalle...,deportes,Torneo Clausura 2024,estadio Alejandro Villanueva
16593,2024-08-06,biobiochile,"A pesar de haber tenido un mayo muy lluvioso, ...",https://www.biobiochile.cl/noticias/servicios/...,¿Seguirán las lluvias? Así será el tiempo en a...,53.565.419,pesar de haber tenido un mayo muy lluvioso jul...,economia,vientos de hasta 125 km/h,"región Metropolitana,"
14156,2024-08-09,centralweb,"En el mundo del Pop Indie y Pop Alternativo, M...",https://www.centralweb.cl/desde-argentina-mart...,"Desde Argentina: Marte presenta «Caótico», su ...",53.558.450,En el mundo del Pop Indie y Pop Alternativo MA...,entretenimiento,"Hoy, desde Madrid, celebra el lanzamiento","Madrid,"
2141,2024-08-27,ciperchile,Hoy nuestra principal fuente de financiamiento...,https://www.ciperchile.cl/2024/08/28/japon-mar...,Japón marca un nuevo precedente en la preparac...,53.554.604,Hoy nuestra principal fuente de financiamiento...,ciencia,megaterremoto,Tohoku Oki


In [14]:
display(df_filtered)

,date,media_outlet,text,url,title,id_news,tokenizacions,category,event,address,latitud,longitud
10404,2024-08-15,biobiochile,Los reconocidos guitarristas Horacio Salinas y...,https://www.biobiochile.cl/noticias/artes-y-cu...,Horacio Salinas y Romilio Orellana unen sus tr...,53.878.712,Los reconocido guitarrista Horacio Salinas y R...,entretenimiento,un concierto único.,Sala SCD Plaza Egaña.,NaN,NaN
20026,2024-08-01,lanacion,El Gobierno suspendió las clases en todos los...,https://www.lanacion.cl/suspenden-clases-en-la...,Suspenden clases en la RM y otras nueve region...,53.412.225,El Gobierno suspendió las clase en todo los es...,educacion,estragos causados por el sistema frontal.,Región de Coquimbo,-30.754665,-70.900554
2043,2024-08-27,latercera,"El delegado de la Región Metropolitana, Gonzal...",https://www.latercera.com/nacional/noticia/se-...,¿Se cierra o no se cierra el Teatro Caupolicán...,53.558.369,El delegado de la Región Metropolitana Gonzalo...,entretenimiento,"concierto,","Teatro Caupolicán,",-33.456217,-70.649239
11508,2024-08-13,biobiochile,"Dos adolescentes de 15 y 16 años, este último ...",https://www.biobiochile.cl/noticias/nacional/r...,Detienen a dos adolescentes por robo de especi...,53.877.607,Dos adolescente de 15 y 16 año este último con...,ciencia,"robo con violencia,",calle Lynch a la altura del Destacamento Tuca...,NaN,NaN
8398,2024-08-18,biobiochile,"Según el pronóstico del tiempo, se esperan llu...",https://www.biobiochile.cl/noticias/servicios/...,"Anuncian lluvias, granizos, vientos sobre 70 k...",53.560.642,Según el pronóstico del tiempo se esperan lluv...,medio_ambiente,"tormentas eléctricas,","zona centro y sur,",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13499,2024-08-10,elcomercio,"/ ElucidarioProvee contexto, definición y deta...",https://elcomercio.pe/respuestas/que/la-respue...,La respuesta de Carlos Zambrano a las fuertes ...,53.406.006,ElucidarioProvee contexto definición y detalle...,deportes,Torneo Clausura 2024,estadio Alejandro Villanueva,-12.068504,-77.022927
16593,2024-08-06,biobiochile,"A pesar de haber tenido un mayo muy lluvioso, ...",https://www.biobiochile.cl/noticias/servicios/...,¿Seguirán las lluvias? Así será el tiempo en a...,53.565.419,pesar de haber tenido un mayo muy lluvioso jul...,economia,vientos de hasta 125 km/h,"región Metropolitana,",-33.573934,-70.620552
14156,2024-08-09,centralweb,"En el mundo del Pop Indie y Pop Alternativo, M...",https://www.centralweb.cl/desde-argentina-mart...,"Desde Argentina: Marte presenta «Caótico», su ...",53.558.450,En el mundo del Pop Indie y Pop Alternativo MA...,entretenimiento,"Hoy, desde Madrid, celebra el lanzamiento","Madrid,",40.416705,-3.703582
2141,2024-08-27,ciperchile,Hoy nuestra principal fuente de financiamiento...,https://www.ciperchile.cl/2024/08/28/japon-mar...,Japón marca un nuevo precedente en la preparac...,53.554.604,Hoy nuestra principal fuente de financiamiento...,ciencia,megaterremoto,Tohoku Oki,NaN,NaN


In [13]:
from geopy.exc import GeocoderTimedOut

# Inicializar el geocodificador
geolocator = Nominatim(user_agent="desafio2")

def clean_address(address):
    """Limpiar, normalizar y convertir a minúsculas la dirección."""
    return address.strip().replace("..", "").replace(".", "").replace(",", "").replace("  ", " ")


def geocode_address(address, retries=3):
    """Geocodificar una dirección para obtener latitud y longitud."""
    if address and address != "no encontrada":
        address = clean_address(address)

        try:
            location = geolocator.geocode(address)
            if location:
                print(f"Geocodificación exitosa para {address}: ({location.latitude}, {location.longitude})")
                return location.latitude, location.longitude
            else:
                print(f"No se encontró ubicación para {address}.")
                return None, None
        except GeocoderTimedOut:
            if retries > 0:
                print(f"Tiempo de espera agotado para {address}. Reintentando ({retries} intentos restantes)...")
                return geocode_address(address, retries=retries-1)
            else:
                print(f"Falló la geocodificación para {address} después de varios intentos.")
                return None, None
        except Exception as e:
            print(f"Error al geocodificar {address}: {e}")
            return None, None
    return None, None  # Retornar None si no hay dirección válida

# Aplicar la geocodificación con manejo de progreso
tqdm.pandas(desc="Geocodificando direcciones")
df_filtered[['latitud', 'longitud']] = df_filtered['address'].progress_apply(lambda x: pd.Series(geocode_address(x)))

# Seleccionar las columnas finales para el CSV de salida
output_df = df_filtered[['id_news', 'event', 'category', 'address', 'latitud', 'longitud']]

# Mostrar el DataFrame de salida
display(output_df)

# Guardar el DataFrame de salida en un archivo CSV
output_df.to_csv('output.csv', index=False)



Geocodificando direcciones:   0%|          | 2/500 [00:00<03:45,  2.20it/s]

No se encontró ubicación para Sala SCD Plaza Egaña.


Geocodificando direcciones:   1%|          | 3/500 [00:01<03:43,  2.23it/s]

Geocodificación exitosa para Región de Coquimbo: (-30.7546652, -70.9005536)


Geocodificando direcciones:   1%|          | 4/500 [00:01<03:50,  2.15it/s]

...

Geocodificando direcciones: 100%|██████████| 500/500 [05:03<00:00,  1.62it/s]

No se encontró ubicación para Tohoku Oki.


Geocodificando direcciones: 100%|██████████| 500/500 [05:03<00:00,  1.65it/s]

Geocodificación exitosa para Arequipa: (-16.3988667, -71.5369607)


,id_news,event,category,address,latitud,longitud
10404,53.878.712,un concierto único.,entretenimiento,Sala SCD Plaza Egaña.,NaN,NaN
20026,53.412.225,estragos causados por el sistema frontal.,educacion,Región de Coquimbo,-30.754665,-70.900554
2043,53.558.369,"concierto,",entretenimiento,"Teatro Caupolicán,",-33.456217,-70.649239
11508,53.877.607,"robo con violencia,",ciencia,calle Lynch a la altura del Destacamento Tuca...,NaN,NaN
8398,53.560.642,"tormentas eléctricas,",medio_ambiente,"zona centro y sur,",NaN,NaN
...,...,...,...,...,...,...
13499,53.406.006,Torneo Clausura 2024,deportes,estadio Alejandro Villanueva,-12.068504,-77.022927
16593,53.565.419,vientos de hasta 125 km/h,economia,"región Metropolitana,",-33.573934,-70.620552
14156,53.558.450,"Hoy, desde Madrid, celebra el lanzamiento",entretenimiento,"Madrid,",40.416705,-3.703582
2141,53.554.604,megaterremoto,ciencia,Tohoku Oki,NaN,NaN
